In [1]:
import smtplib

In [2]:
from email.mime.text import MIMEText

In [3]:
# 시스템 라이브러리 활용 패키지
import os

# 이메일 메시지에 형식을 활용하기 위한 패키지
from email.mime.multipart import MIMEMultipart

# 이메일 메시지 프로토콜 처리기(엔코더)
from email import encoders

# 텍스트 형식 메일
from email.mime.text import MIMEText

# 이미지 형식 메일
from email.mime.image import MIMEImage

# 오디오 형식 메일
from email.mime.audio import MIMEAudio

# 이 모든 정보들을 관장하는 객체
from email.mime.base import MIMEBase

In [4]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
        # audio, video는 별도 설정 없이 진행하도록 함
        elif key == 'audio':
            print("음원은 지원하지 않습니다!")
        elif key == 'video':
            print("영상은 지원하지 않습니다!")
        # 그 외의 형식
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
                
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment', 
                       filename = os.path.basename(value['filename']))

        # 파일 첨부!
        multi.attach(msg)
        
    return multi

In [5]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 설정 및 시작 (보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
        
        # 이메일 전송
        # 메시지 전송시 반드시 as_string()으로 변환하여 전송해야함
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공!!!')
        else:
            print(res)

In [6]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json
import requests
import datetime
import os

In [7]:
import requests

url = "https://kauth.kakao.com/oauth/token"

#data = {
#    "grant_type" : "authorization_code",
#    "client_id" : "REST_API_키",
#    "redirect_uri" : "https://localhost.com",
#    "code" : "발급받은 토큰값"
#}

data = {
    "grant_type" : "authorization_code",
    "client_id" : "",
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

{'access_token': 'CDihr30VdvSRWxHBQVYPcCIwW6fUR3w5_He18AopcFAAAAF9awwQmw', 'token_type': 'bearer', 'refresh_token': '1HccIDnNTrF5OWhx0cwTEVW3RZ4_7jxr7T8JWAopcFAAAAF9awwQmg', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}


In [8]:
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"
APP_KEY_SAVED_FILE = "res/kakao_msg/app_key"

In [9]:
def save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens):
    with open(KAKAO_TOKEN_SAVED_FILE, "w") as fp:
        json.dump(tokens, fp)

In [10]:
save_tokens(KAKAO_TOKEN_SAVED_FILE,tokens)

In [11]:
# 초기 한 번만 수행함(REST APP KEY 저장 목적)
import pickle

app_key = ""

f = open(APP_KEY_SAVED_FILE, 'wb')
pickle.dump(app_key, f)
f.close()

In [12]:
# REST APP KEY 저장 잘 되었는지 확인하는 목적
f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

print()

In [13]:
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [14]:
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "." + now
        os.rename(filename, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/spring2pythonRequestWithData', methods = ['GET', 'POST'])
def spring2pythonRequestWithData():
    
    params = json.loads(request.get_data(), encoding='utf-8')
    print("params: ", params)
    
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_string = ""
    name = ""
    board_no = ""
    
    ##################### 개조 구간 시작 #####################
    #params_string = ""
    #for key in params.keys():
    #    params_string += 'key: {}, value: {}<br>'.format(key, params[key])
    
    values = []
    for key in params.keys():
        values.append(params[key])
        
    print("values:", values)
    board_no = values[0]
    name =  values[1]
    
    print("게시판번호:", board_no)
    print("작성자:", name)
        
    
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)
    print("tokens: ", tokens)

    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : "글쓴이{1}의{0}번 게시글이 신고되었습니다 확인부탁드립니다.".format(board_no,name),
            "link" : {
                "web_url" : "http://localhost:8080/boardReadPage/{0}".format(board_no)
            }
        })
    }

    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
    
    if response.status_code == 401:
        tokens = update_tokens(app_key, KAKAO_TOKEN_SAVED_FILE)
        
    print("tokens: ", tokens)
    
    ##################### 개조 구간 끄읕 #####################
    
    return jsonify(params_string)

@app.route('/sendNaverEamil', methods = ['GET', 'POST'])
def sendNaverEamil():
    params = json.loads(request.get_data(), encoding='utf-8')
    print("넘어온값: ", params)
    #넘어온값이 없으면
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_number= ""
    Authentication_num = ""
    values = []
    for key in params.keys():
        values.append(params[key])
    
    
    Authentication_num = values[0]
    print("인증번호:", Authentication_num)
    
    smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": "tkddn5678@naver.com",
    "smtp_user_pw": "qweasdzxc1212~",
    "smtp_port": 587
    })

    title = "[EVS]인증번호"
    content =  str(values[0])
    sender = smtp_info['smtp_user_id']
    receiver = smtp_info['smtp_user_id']

    msg = MIMEText(_text = content, _charset = "utf-8")

    msg['Subject'] = title
    msg['From'] = sender
    msg['To'] = receiver
    
    send_email(smtp_info, msg)
    
    
    
    return params_number
    

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2021 18:35:39] "OPTIONS /sendNaverEamil HTTP/1.1" 200 -
<ipython-input-15-748277e281d8>:68: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')


넘어온값:  {'randomNum2': 72974}
인증번호: 72974


127.0.0.1 - - [29/Nov/2021 18:35:39] "POST /sendNaverEamil HTTP/1.1" 200 -


이메일 전송 성공!!!


<ipython-input-15-748277e281d8>:7: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')
127.0.0.1 - - [29/Nov/2021 18:36:17] "POST /spring2pythonRequestWithData HTTP/1.1" 200 -


params:  {'boardNo': 5, 'memberId': 'tkddn1212'}
values: [5, 'tkddn1212']
게시판번호: 5
작성자: tkddn1212
tokens:  {'access_token': 'CDihr30VdvSRWxHBQVYPcCIwW6fUR3w5_He18AopcFAAAAF9awwQmw', 'token_type': 'bearer', 'refresh_token': '1HccIDnNTrF5OWhx0cwTEVW3RZ4_7jxr7T8JWAopcFAAAAF9awwQmg', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}
200
tokens:  {'access_token': 'CDihr30VdvSRWxHBQVYPcCIwW6fUR3w5_He18AopcFAAAAF9awwQmw', 'token_type': 'bearer', 'refresh_token': '1HccIDnNTrF5OWhx0cwTEVW3RZ4_7jxr7T8JWAopcFAAAAF9awwQmg', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}
